# Machine Learning Code for Wine Quality Project

## First step is to load in the needed libraries for the project.  For this particular project we are using pandas for general dataframe manipulation, psycopg2 for PostgreSQL database connection, and train_test_split and LinearRegression from sklearn for model creation and prediction

In [1]:
#import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import psycopg2
import numpy as np

### Use psycopg2 to connect to external database

In [2]:
#connect to PostgreSQL database
conn = psycopg2.connect(
    host=XXXX,
    database = XXXX,
    user = XXXX,
    password = XXXX)


## Due to database storage limitations through AWS, combined with the overall size of the initial training dataset, the training data obtained through Kaggle will be loaded directly into the machine learning code via csv file and processed accordingly.

In [3]:
#read in training data for model creation from local csv file
file = 'winequality_training_data2.csv'
wine_df = pd.read_csv(file)

#test to make sure dataframe was set up correctly
wine_df.head()

,color,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,6.2,0.450,0.26,4.4,0.063,63.0,206.0,0.9940,3.27,0.52,9.8,4
1,white,9.8,0.360,0.46,10.5,NaN,4.0,83.0,0.9956,2.89,0.30,10.1,4
2,white,5.5,0.485,0.00,1.5,0.065,8.0,103.0,0.9940,3.63,0.40,9.7,4
3,white,6.4,0.595,0.14,5.2,0.058,15.0,97.0,0.9951,3.38,0.36,9.0,4
4,white,7.6,0.480,0.37,0.8,0.037,4.0,100.0,0.9902,3.03,0.39,11.4,4


### Primary cleanup of datafile will be completed for demonstration purposes as well as redundancy using Python code.

In [4]:
#drop any remaining null rows
wine_df = wine_df.dropna()

#drop any null columns
wine_df = wine_df.dropna(axis='columns', how='all')

In [5]:
#additional dataframe manipulation to covert string values for wine colors to integers
wine_df = pd.get_dummies(wine_df, columns=['color'])

#verify updated dataframe
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color_red,color_white
0,6.2,0.450,0.26,4.4,0.063,63.0,206.0,0.9940,3.27,0.52,9.8,4,0,1
2,5.5,0.485,0.00,1.5,0.065,8.0,103.0,0.9940,3.63,0.40,9.7,4,0,1
3,6.4,0.595,0.14,5.2,0.058,15.0,97.0,0.9951,3.38,0.36,9.0,4,0,1
4,7.6,0.480,0.37,0.8,0.037,4.0,100.0,0.9902,3.03,0.39,11.4,4,0,1
5,7.2,0.320,0.47,5.1,0.044,19.0,65.0,0.9910,3.03,0.41,12.6,4,0,1


### Multiple potential models were attempted in order to identify the one that would be most fitting with this dataset.  At one point, it was determined that the RandomForestClassifier was the best suited, but upon further review of the resulting data, this was discovered to be in error.  The classifier model was not calculating predicted quailty scores correctly.   Further testing of various training models was completed and multiple linear regression was determined to produce the highest results. To accomplish this, we established the features to be used in the multiple linear regression model and trained the model using the uploaded training dataset.

In [6]:
#establish features to be used in the linear regression model
X = wine_df[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol','color_red','color_white']]
y = wine_df['quality']
             
#build linear regression model 
model = LinearRegression()

#fit training data to model
model.fit(X, y)


LinearRegression()

###  Generally speaking, an R2 score of 70 or higher would be most desirable.  However, due to the nature of the data that we are working with, the resulting R2 score was the best available option.  The model was used against smaller datasets for prediction to assess accuracy and was found to be within acceptable limits

In [7]:
#calculate accuracy score
score = model.score(X, y)

print(f'R2 Score: {score}')

R2 Score: 0.5612103200433791


### It was determined that our data to be tested would reflect 20 potential wines created by our winery.  To create a reasonable dataset, we took a sample of approximately 20 red and 20 white wines covering a range of quality scores from 3 to 9.  Using the individual feature values of those wines, we created a random number generator program (Wine Test File Creator) that would create random results for each feature within the highest and lowest value for each feature within that dataset.  The creator file then created a sample set csv file which was then loaded into this code for testing and calculating quality scores.

In [8]:
#load test data produced by our wine
file = 'winequality-sample-set.csv'
test_df = pd.read_csv(file)

test_df = pd.get_dummies(test_df, columns=['color'])
y = test_df['wine_id']

test_df = test_df.drop(columns='wine_id')
predictions = np.round(model.predict(test_df))

print(predictions)

[5. 8. 5. 5. 7. 6. 6. 6. 5. 7. 5. 7. 6. 6. 9. 8. 6. 6. 5. 6.]


In [9]:
#create final dataframe with calculated data
final_set = test_df

final_set['quality'] = predictions

final_set.insert(0, 'wine_id', '')
final_set['wine_id'] = y

#show final set
final_set.head(20)

,wine_id,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,color_red,color_white,quality
0,w001,11.1,0.793,0.40,2.4,0.088,7,84,0.9968,3.71,0.87,10.0,1,0,5.0
1,w002,11.2,0.301,0.52,8.8,0.044,14,95,0.9968,3.02,1.15,10.0,1,0,8.0
2,w003,6.5,0.520,0.01,4.1,0.049,16,137,0.9988,3.68,0.64,11.7,1,0,5.0
3,w004,9.2,0.749,0.22,2.5,0.049,9,23,0.9959,3.13,0.80,9.2,1,0,5.0
4,w005,8.5,0.194,0.26,7.4,0.063,32,145,0.9994,3.36,0.89,12.1,1,0,7.0
5,w006,6.0,0.748,0.36,6.5,0.063,29,106,0.9965,3.87,0.82,10.9,1,0,6.0
6,w007,10.8,0.636,0.14,9.5,0.084,15,128,0.9983,3.58,0.81,12.2,1,0,6.0
7,w008,6.2,0.632,0.88,2.7,0.085,18,58,0.9957,3.99,0.78,10.6,1,0,6.0
8,w009,10.9,0.669,0.55,4.2,0.059,24,179,0.9995,3.41,0.76,10.5,1,0,5.0
9,w010,5.3,0.563,0.89,7.4,0.038,31,68,0.9974,3.98,0.53,13.4,1,0,7.0


### Separated out the white and red wines for upload into AWS database

In [10]:
#separate out white and red datasets for upload to database
white_df = final_set.loc[final_set['color_white'] == 1]
red_df = final_set.loc[final_set['color_red'] == 1]

In [11]:
#verify white wine file
white_df.head(10)

,wine_id,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,color_red,color_white,quality
10,w011,8.1,0.773,0.30,1.2,0.045,17,126,0.9984,3.26,1.06,13.1,0,1,5.0
11,w012,11.0,0.472,0.76,6.2,0.035,26,95,0.9967,3.06,0.74,11.4,0,1,7.0
12,w013,6.7,0.703,0.78,4.1,0.027,28,39,0.9989,3.29,1.03,11.2,0,1,6.0
13,w014,9.5,0.770,0.83,8.4,0.069,18,22,0.9972,3.53,0.51,12.2,0,1,6.0
14,w015,11.6,0.503,0.81,10.0,0.056,24,50,0.9953,3.65,0.82,12.5,0,1,9.0
15,w016,10.3,0.341,0.70,10.1,0.081,15,91,0.9957,3.45,0.98,9.4,0,1,8.0
16,w017,11.0,0.777,0.54,7.8,0.068,13,63,0.9988,3.90,0.75,12.3,0,1,6.0
17,w018,7.2,0.342,0.34,4.8,0.062,7,123,0.9956,3.24,0.68,10.9,0,1,6.0
18,w019,12.6,0.749,0.05,6.5,0.063,20,79,0.9986,3.43,0.78,9.2,0,1,5.0
19,w020,7.6,0.514,0.76,5.2,0.035,19,128,0.9970,3.93,0.57,12.9,0,1,6.0


In [12]:
#verify red wine file
red_df.head(10)

,wine_id,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,color_red,color_white,quality
0,w001,11.1,0.793,0.40,2.4,0.088,7,84,0.9968,3.71,0.87,10.0,1,0,5.0
1,w002,11.2,0.301,0.52,8.8,0.044,14,95,0.9968,3.02,1.15,10.0,1,0,8.0
2,w003,6.5,0.520,0.01,4.1,0.049,16,137,0.9988,3.68,0.64,11.7,1,0,5.0
3,w004,9.2,0.749,0.22,2.5,0.049,9,23,0.9959,3.13,0.80,9.2,1,0,5.0
4,w005,8.5,0.194,0.26,7.4,0.063,32,145,0.9994,3.36,0.89,12.1,1,0,7.0
5,w006,6.0,0.748,0.36,6.5,0.063,29,106,0.9965,3.87,0.82,10.9,1,0,6.0
6,w007,10.8,0.636,0.14,9.5,0.084,15,128,0.9983,3.58,0.81,12.2,1,0,6.0
7,w008,6.2,0.632,0.88,2.7,0.085,18,58,0.9957,3.99,0.78,10.6,1,0,6.0
8,w009,10.9,0.669,0.55,4.2,0.059,24,179,0.9995,3.41,0.76,10.5,1,0,5.0
9,w010,5.3,0.563,0.89,7.4,0.038,31,68,0.9974,3.98,0.53,13.4,1,0,7.0


In [14]:
#create csv files for white and red wine
red_df.to_csv('red_wine_file.csv')
white_df.to_csv('white_wine_file.csv')


In [ ]:
#writing results to sql database
white_df.to_sql('wine_white', con=conn, if_exists='append', index=False)
red_df.to_sql('wine_red', con=conn, if_exists='append', index=False)